In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive
!mkdir CS725_Project_FINAL
%cd /content/drive/MyDrive/CS725_Project_FINAL/

/content/drive/MyDrive
/content/drive/MyDrive/CS725_Project_FINAL


In [ ]:
!git clone https://github.com/abreorussel/CS725-retinal-vessel-segmentation.git

Cloning into 'CS725-retinal-vessel-segmentation'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 87 (delta 49), reused 62 (delta 24), pack-reused 0 (from 0)
Receiving objects: 100% (87/87), 27.97 KiB | 2.80 MiB/s, done.
Resolving deltas: 100% (49/49), done.


In [ ]:
!pip install kaggle

In [ ]:
%cd CS725-retinal-vessel-segmentation
!mkdir retinal-blood-vessels
%cd /content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels

/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation
/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels


In [ ]:
!mkdir ~/.kaggle

! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdallahwagih/retina-blood-vessel")

print("Path to dataset files:", path)

100%|██████████| 32.9M/32.9M [00:03<00:00, 10.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/abdallahwagih/retina-blood-vessel/versions/1


In [ ]:
%cd /root/.cache/kagglehub/datasets/abdallahwagih/retina-blood-vessel/versions/1

/root/.cache/kagglehub/datasets/abdallahwagih/retina-blood-vessel/versions/1


In [ ]:
%cd /content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/

/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation


In [ ]:
import os
import torch


def mkdir(*paths):
    for path in paths:
        if not os.path.exists(path):
            os.makedirs(path, exist_ok=True)

ROOT_DIR = os.path.dirname(os.path.abspath('__file__'))
DATA_DIR = os.path.join(ROOT_DIR, 'retinal-blood-vessels')
CKPT_DIR = os.path.join(ROOT_DIR, 'checkpoints')
LOG_DIR = os.path.join(ROOT_DIR, 'logs')
TRAIN_LOG_DIR = os.path.join(LOG_DIR, 'train')
VAL_LOG_DIR = os.path.join(LOG_DIR, 'val')

IMGS_DIR = os.path.join(DATA_DIR, 'imgs')
LABELS_DIR = os.path.join(DATA_DIR, 'labels')

TRAIN_IMGS_DIR = os.path.join(IMGS_DIR, 'train')
VAL_IMGS_DIR = os.path.join(IMGS_DIR, 'val')
# TEST_IMGS_DIR = os.path.join(IMGS_DIR, 'test')

TRAIN_LABELS_DIR = os.path.join(LABELS_DIR, 'train')
VAL_LABELS_DIR = os.path.join(LABELS_DIR, 'val')
# TEST_LABELS_DIR = os.path.join(LABELS_DIR, 'test')

mkdir(
    CKPT_DIR, LOG_DIR, TRAIN_LOG_DIR, VAL_LOG_DIR,
    IMGS_DIR, LABELS_DIR, TRAIN_IMGS_DIR, VAL_IMGS_DIR,
    TRAIN_LABELS_DIR, VAL_LABELS_DIR,
    )

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
VAL_IMGS_DIR

'/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels/imgs/val'

In [ ]:
DATASET_DIR = "/root/.cache/kagglehub/datasets/abdallahwagih/retina-blood-vessel/versions/1/Data"

## MOVE the TEST Data

In [ ]:
import shutil
import os

TEST_IMGS = os.path.join(DATASET_DIR , "test/image")
TEST_IMGS_DIR = os.path.join(DATA_DIR , "imgs/test")

TEST_LABELS = os.path.join(DATASET_DIR , "test/mask")
TEST_LABELS_DIR = os.path.join(DATA_DIR , "labels/test")

shutil.copytree(TEST_IMGS, TEST_IMGS_DIR)
shutil.copytree(TEST_LABELS, TEST_LABELS_DIR)

'/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels/labels/test'

In [ ]:
TRAIN_LABELS = os.path.join(DATASET_DIR , "train/mask")
TRAIN_DATA_LABELS_DIR = os.path.join(DATA_DIR , "labels/train-data")

TRAIN_IMGS = os.path.join(DATASET_DIR , "train/image")
TRAIN_DATA_IMGS_DIR = os.path.join(DATA_DIR , "imgs/train-data")

shutil.copytree(TRAIN_LABELS, TRAIN_DATA_LABELS_DIR)
shutil.copytree(TRAIN_IMGS, TRAIN_DATA_IMGS_DIR)

'/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels/imgs/train-data'

In [ ]:

import os
import random

import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

TRAIN_LABELS_DIR = os.path.join(DATA_DIR , "labels/train")
TRAIN_IMGS_DIR = os.path.join(DATA_DIR , "imgs/train")

TRAIN_LABELS_DIR
TRAIN_IMGS_DIR

train_img_path = TRAIN_DATA_IMGS_DIR
train_label_path = TRAIN_DATA_LABELS_DIR

image_mask_pairs = list()


image_files = sorted(os.listdir(train_img_path))
mask_files = sorted(os.listdir(train_label_path))


for image_file, mask_file in zip(image_files, mask_files):
  image_path = os.path.join(train_img_path, image_file)
  mask_path = os.path.join(train_label_path, mask_file)
  image_mask_pairs.append((image_path, mask_path))

# Shuffle and Split Data
random.shuffle(image_mask_pairs)
print(len(image_mask_pairs))
train_list = image_mask_pairs[:60]
val_list = image_mask_pairs[60:]



# # Save splitted files

for idx, (img_path, label_path) in enumerate(train_list, 1):
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # Load the image
  label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)  # Load the label

  if img is None or label is None:
      print(f"Error loading image or label for {img_path} and {label_path}")
      continue

  img_dst = os.path.join(TRAIN_IMGS_DIR, f'{idx:02}.png')
  label_dst = os.path.join(TRAIN_LABELS_DIR, f'{idx:02}.png')
  cv2.imwrite(img_dst, img)
  cv2.imwrite(label_dst, label)



for idx, (img_path, label_path) in enumerate(val_list, 1):
  img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # Load the image
  label = cv2.imread(label_path, cv2.IMREAD_UNCHANGED)  # Load the label

  if img is None or label is None:
      print(f"Error loading image or label for {img_path} and {label_path}")
      continue

  img_dst = os.path.join(VAL_IMGS_DIR, f'{idx:02}.png')
  label_dst = os.path.join(VAL_LABELS_DIR, f'{idx:02}.png')
  cv2.imwrite(img_dst, img)
  cv2.imwrite(label_dst, label)

80


In [ ]:
VAL_IMGS_DIR

'/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels/imgs/val'

In [ ]:
IMGS_DIR

'/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/retinal-blood-vessels/imgs'

In [ ]:
%cd /content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation

!python train.py

/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation
2024-11-25 10:00:33.324936: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 10:00:33.342867: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 10:00:33.364613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 10:00:33.371121: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been regis

In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 401 bytes | 16.00 KiB/s, done.
From https://github.com/abreorussel/CS725-retinal-vessel-segmentation
   ee66a49..0b2bb88  main       -> origin/main
Updating ee66a49..0b2bb88
Fast-forward
 train.py | 22 +++++++++++-----------
 1 file changed, 11 insertions(+), 11 deletions(-)


In [ ]:
%cd /content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation

!python eval_new.py

/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation
/content/drive/MyDrive/CS725_Project_FINAL/CS725-retinal-vessel-segmentation/utils.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues 